# Scraping APIs

A site with an API (Application Programming Interface) wants you to scrape it.

Examples abound:

* <a href="https://www.census.gov/data/developers/data-sets.html">U.S. Census APIs</a>
* <a href="https://apps.fas.usda.gov/opendataweb/home">US Agriculture Commodities and Exports</a>
* <a href="https://www.federalregister.gov/developers/documentation/api/v1">Federal Register</a>
* <a href="https://developer.dol.gov/beginners-guide/">Labor Department</a>
* <a href="https://www.eia.gov/">U.S. Energy Information Department</a>

Government sites tend  ```CSVs``` for download but their APIs offer more detailed options for data. They are not trying to hide the data.

Private sites might have APIs, but often charge heafty prices for access beyond a basic number of downloads.

The toughest/hardest part of scraping an API is that they ***ALL HAVE DIFFERENT INSTRUCTIONS*** on how to tap their data.

Today, we'll explore different APIs that each build a different skill:

1. Census health data – **building a simple API call.**
2. USDA commodities exports – **using an API key and targeting specific commodities over several years.**
3. Federal Register – **tapping search terms.**
4. Energy Information Administration – **dealing with pagination.**

What they all have in common:

1. a base url
2. a query string
3. tied together with a query character ```?```
4. an API key.

Combined together these are known as an ```API endpoint```.

You make an ```API call``` (a request) using the ```API endpoint```.




In [1]:
## import libraries
import requests
import pandas as pd
# from icecream import ic



## 1. Census health data – **building a simple API call.**

- <a href="https://www.census.gov/data/developers/data-sets/Health-Insurance-Statistics.html">Census health landing page</a>
- List of <a href="https://api.census.gov/data/timeseries/healthins/sahie/variables.html">possible variables</a>

We want to create a dataframe with the following info for every state in 2021:

1. Total number insured
2. Percent insured
3. Total number uninsured
4. Percent uninsured
5. by Race

## the parts to build your API call.

In [20]:
## create a dictionary to know what codes mean
## all variables are 90% upper bound
target_dict = {
    "NIC_UB90": "total_insured",
    "PCTIC_PT": "pct_insured",
    "NUI_UB90": "total_uninsured",
    "PCTUI_PT": "pct_uninsured",
    "RACECAT": "race_cat",
    "RACE_DESC": "race_description",
    "STABREV": "state",
    "STATE": "fips_code"
    
}

In [21]:
## pull my keys

list(target_dict.keys())

['NIC_UB90',
 'PCTIC_PT',
 'NUI_UB90',
 'PCTUI_PT',
 'RACECAT',
 'RACE_DESC',
 'STABREV',
 'STATE']

In [22]:
target_dict.keys()

dict_keys(['NIC_UB90', 'PCTIC_PT', 'NUI_UB90', 'PCTUI_PT', 'RACECAT', 'RACE_DESC', 'STABREV', 'STATE'])

In [23]:
## pull values

list(target_dict.values())

['total_insured',
 'pct_insured',
 'total_uninsured',
 'pct_uninsured',
 'race_cat',
 'race_description',
 'state',
 'fips_code']

In [24]:
## format into api query format
target_vars = ",".join(target_dict.keys())
target_vars

'NIC_UB90,PCTIC_PT,NUI_UB90,PCTUI_PT,RACECAT,RACE_DESC,STABREV,STATE'

In [25]:
## other targets


In [26]:
## base url + year

base_url= "https://api.census.gov/data/timeseries/healthins/sahie?get="
target_year = "2021"

In [27]:
## create query string
query_string = f"{target_vars}&for=state:*&time={target_year}"
query_string

'NIC_UB90,PCTIC_PT,NUI_UB90,PCTUI_PT,RACECAT,RACE_DESC,STABREV,STATE&for=state:*&time=2021'

In [28]:
## create full API call
endpoint = base_url + query_string
endpoint

'https://api.census.gov/data/timeseries/healthins/sahie?get=NIC_UB90,PCTIC_PT,NUI_UB90,PCTUI_PT,RACECAT,RACE_DESC,STABREV,STATE&for=state:*&time=2021'

In [29]:
## get response
response = requests.get(endpoint)

In [31]:
## turn response into json
data = response.json()
data

[['NIC_UB90',
  'PCTIC_PT',
  'NUI_UB90',
  'PCTUI_PT',
  'RACECAT',
  'RACE_DESC',
  'STABREV',
  'STATE',
  'time',
  'state'],
 ['3563072',
  '88.3',
  '484434',
  '11.7',
  '0',
  'All Races',
  'AL',
  '01',
  '2021',
  '01'],
 ['534609',
  '86.8',
  '84822',
  '13.2',
  '0',
  'All Races',
  'AK',
  '02',
  '2021',
  '02'],
 ['5035973',
  '86.9',
  '777974',
  '13.1',
  '0',
  'All Races',
  'AZ',
  '04',
  '2021',
  '04'],
 ['2169221',
  '88.9',
  '278814',
  '11.1',
  '0',
  'All Races',
  'AR',
  '05',
  '2021',
  '05'],
 ['30129212',
  '91.9',
  '2681093',
  '8.1',
  '0',
  'All Races',
  'CA',
  '06',
  '2021',
  '06'],
 ['4420952',
  '90.7',
  '467216',
  '9.3',
  '0',
  'All Races',
  'CO',
  '08',
  '2021',
  '08'],
 ['2711824',
  '94.0',
  '182120',
  '6.0',
  '0',
  'All Races',
  'CT',
  '09',
  '2021',
  '09'],
 ['728780',
  '93.1',
  '57747',
  '6.9',
  '0',
  'All Races',
  'DE',
  '10',
  '2021',
  '10'],
 ['530990',
  '96.3',
  '23332',
  '3.7',
  '0',
  'All Race

In [37]:
## create dataframe
df = pd.DataFrame(data[1:], columns = data[0])
# df = pd.DataFrame(data)
df

,NIC_UB90,PCTIC_PT,NUI_UB90,PCTUI_PT,RACECAT,RACE_DESC,STABREV,STATE,time,state
0,3563072,88.3,484434,11.7,0,All Races,AL,01,2021,01
1,534609,86.8,84822,13.2,0,All Races,AK,02,2021,02
2,5035973,86.9,777974,13.1,0,All Races,AZ,04,2021,04
3,2169221,88.9,278814,11.1,0,All Races,AR,05,2021,05
4,30129212,91.9,2681093,8.1,0,All Races,CA,06,2021,06
...,...,...,...,...,...,...,...,...,...,...
403,301181,92.4,26751,7.6,7,"Two or More Races, not Hispanic or Latino",VA,51,2021,51
404,422280,94.0,29465,6.0,7,"Two or More Races, not Hispanic or Latino",WA,53,2021,53
405,38653,92.9,3386,7.1,7,"Two or More Races, not Hispanic or Latino",WV,54,2021,54
406,133676,94.0,9510,6.0,7,"Two or More Races, not Hispanic or Latino",WI,55,2021,55


In [38]:
## rename column headers with more meaning full headers
df.rename(columns = target_dict, inplace = True)
df

,total_insured,pct_insured,total_uninsured,pct_uninsured,race_cat,race_description,state,fips_code,time,state
0,3563072,88.3,484434,11.7,0,All Races,AL,01,2021,01
1,534609,86.8,84822,13.2,0,All Races,AK,02,2021,02
2,5035973,86.9,777974,13.1,0,All Races,AZ,04,2021,04
3,2169221,88.9,278814,11.1,0,All Races,AR,05,2021,05
4,30129212,91.9,2681093,8.1,0,All Races,CA,06,2021,06
...,...,...,...,...,...,...,...,...,...,...
403,301181,92.4,26751,7.6,7,"Two or More Races, not Hispanic or Latino",VA,51,2021,51
404,422280,94.0,29465,6.0,7,"Two or More Races, not Hispanic or Latino",WA,53,2021,53
405,38653,92.9,3386,7.1,7,"Two or More Races, not Hispanic or Latino",WV,54,2021,54
406,133676,94.0,9510,6.0,7,"Two or More Races, not Hispanic or Latino",WI,55,2021,55


In [45]:
df_list = []
for target_year in range(2018,2023):
    print(target_year)
    print(base_url + f"{target_vars}&for=state:*&time={target_year}")
    response = requests.get(base_url + f"{target_vars}&for=state:*&time={target_year}")
    data = response.json()
    df = pd.DataFrame(data[1:], columns = data[0])
    df.rename(columns = target_dict, inplace = True)
    df_list.append(df)

2018
https://api.census.gov/data/timeseries/healthins/sahie?get=NIC_UB90,PCTIC_PT,NUI_UB90,PCTUI_PT,RACECAT,RACE_DESC,STABREV,STATE&for=state:*&time=2018
2019
https://api.census.gov/data/timeseries/healthins/sahie?get=NIC_UB90,PCTIC_PT,NUI_UB90,PCTUI_PT,RACECAT,RACE_DESC,STABREV,STATE&for=state:*&time=2019
2020
https://api.census.gov/data/timeseries/healthins/sahie?get=NIC_UB90,PCTIC_PT,NUI_UB90,PCTUI_PT,RACECAT,RACE_DESC,STABREV,STATE&for=state:*&time=2020
2021
https://api.census.gov/data/timeseries/healthins/sahie?get=NIC_UB90,PCTIC_PT,NUI_UB90,PCTUI_PT,RACECAT,RACE_DESC,STABREV,STATE&for=state:*&time=2021
2022
https://api.census.gov/data/timeseries/healthins/sahie?get=NIC_UB90,PCTIC_PT,NUI_UB90,PCTUI_PT,RACECAT,RACE_DESC,STABREV,STATE&for=state:*&time=2022


In [46]:
len(df_list)

5

In [47]:
df = pd.concat(df_list, ignore_index = True)
df

,total_insured,pct_insured,total_uninsured,pct_uninsured,race_cat,race_description,state,fips_code,time,state
0,3498430,88.1,483417,11.9,0,All Races,AL,01,2018,01
1,551881,86.1,92699,13.9,0,All Races,AK,02,2018,02
2,5043389,87.2,756267,12.8,0,All Races,AZ,04,2018,04
3,2204505,90.4,242681,9.6,0,All Races,AR,05,2018,05
4,30505401,91.7,2812946,8.3,0,All Races,CA,06,2018,06
...,...,...,...,...,...,...,...,...,...,...
1423,12522,86.9,2109,13.1,7,"Two or More Races, not Hispanic or Latino",WY,56,2022,56
1424,4022,82.7,955,17.3,2,"Black or African American alone, not Hispanic ...",WY,56,2022,56
1425,6843590,94.3,423868,5.7,1,"White alone, not Hispanic or Latino",PA,42,2022,42
1426,1036963,93.4,79701,6.6,2,"Black or African American alone, not Hispanic ...",PA,42,2022,42


In [ ]:
## cooy race category column to be converted to description next cell


In [ ]:
## replace with codes


### 2. USDA commodities exports – **using an API key and targeting specific commodities over several years.**

- <a href="https://apps.fas.usda.gov/opendataweb/home">USDA APIs endpoints</a>
- Get an <a href="https://apps.fas.usda.gov/opendataweb/home">API key</a>

We want to create a dataframe with exports between 2020-2022 to all countries for the following commodities:

1. All Wheat
2. Oats
3. Cuts of Beef
4. Cuts of Pork

In [73]:
## find the parts to build your API call

headers = {
    'accept': 'application/json',
    'X-Api-Key': 'lx8NMObcTL9PSZSCZzJyykInlqLEx5WZZKtbGITw',
}

response = requests.get('https://api.fas.usda.gov/api/esr/commodities', headers=headers)

In [78]:
target_commodities = [{'commodityCode': 107, 'commodityName': 'All Wheat', 'unitId': 1},
{'commodityCode': 601, 'commodityName': 'Oats', 'unitId': 1},
{'commodityCode': 1701,
  'commodityName': 'Fresh, Chilled, or Frozen Muscle Cuts of Beef',
  'unitId': 1},
 {'commodityCode': 1702,
  'commodityName': 'Fresh, Chilled, or Frozen Muscle Cuts of Pork',
  'unitId': 1}]

In [88]:
base_url = "https://api.fas.usda.gov/api/esr/exports/commodityCode/"
end_url = "/allCountries/marketYear/"

In [101]:
headers = {
        'accept': 'application/json',
        'X-Api-Key': 'lx8NMObcTL9PSZSCZzJyykInlqLEx5WZZKtbGITw',
        }

all_data = []
for year in range(2020, 2022):
    for target_com  in target_commodities:
        endpoint = f'{base_url}{target_com.get("commodityCode")}{end_url}{year}'
        print(endpoint)
        

        response = requests.get(
        endpoint,
        headers=headers,)
        
        all_data.append(response.json())
        

https://api.fas.usda.gov/api/esr/exports/commodityCode/107/allCountries/marketYear/2020
https://api.fas.usda.gov/api/esr/exports/commodityCode/601/allCountries/marketYear/2020
https://api.fas.usda.gov/api/esr/exports/commodityCode/1701/allCountries/marketYear/2020
https://api.fas.usda.gov/api/esr/exports/commodityCode/1702/allCountries/marketYear/2020
https://api.fas.usda.gov/api/esr/exports/commodityCode/107/allCountries/marketYear/2021
https://api.fas.usda.gov/api/esr/exports/commodityCode/601/allCountries/marketYear/2021
https://api.fas.usda.gov/api/esr/exports/commodityCode/1701/allCountries/marketYear/2021
https://api.fas.usda.gov/api/esr/exports/commodityCode/1702/allCountries/marketYear/2021


In [103]:
all_data

[[{'commodityCode': 107,
   'countryCode': 1220,
   'weeklyExports': 199,
   'accumulatedExports': 199,
   'outstandingSales': 18885,
   'grossNewSales': 11268,
   'currentMYNetSales': 10391,
   'currentMYTotalCommitment': 19084,
   'nextMYOutstandingSales': 0,
   'nextMYNetSales': 0,
   'unitId': 1,
   'weekEndingDate': '2019-06-06T00:00:00'},
  {'commodityCode': 107,
   'countryCode': 2010,
   'weeklyExports': 71970,
   'accumulatedExports': 71970,
   'outstandingSales': 688745,
   'grossNewSales': 365035,
   'currentMYNetSales': 90031,
   'currentMYTotalCommitment': 760715,
   'nextMYOutstandingSales': 0,
   'nextMYNetSales': 0,
   'unitId': 1,
   'weekEndingDate': '2019-06-06T00:00:00'},
  {'commodityCode': 107,
   'countryCode': 2050,
   'weeklyExports': 3300,
   'accumulatedExports': 3300,
   'outstandingSales': 15600,
   'grossNewSales': 8800,
   'currentMYNetSales': 300,
   'currentMYTotalCommitment': 18900,
   'nextMYOutstandingSales': 0,
   'nextMYNetSales': 0,
   'unitId': 1

In [104]:
pd.DataFrame(all_data)

,0,1,2,3,4,5,6,7,8,9,...,4076,4077,4078,4079,4080,4081,4082,4083,4084,4085
0,"{'commodityCode': 107, 'countryCode': 1220, 'w...","{'commodityCode': 107, 'countryCode': 2010, 'w...","{'commodityCode': 107, 'countryCode': 2050, 'w...","{'commodityCode': 107, 'countryCode': 2110, 'w...","{'commodityCode': 107, 'countryCode': 2150, 'w...","{'commodityCode': 107, 'countryCode': 2190, 'w...","{'commodityCode': 107, 'countryCode': 2230, 'w...","{'commodityCode': 107, 'countryCode': 2250, 'w...","{'commodityCode': 107, 'countryCode': 2410, 'w...","{'commodityCode': 107, 'countryCode': 2470, 'w...",...,None,None,None,None,None,None,None,None,None,None
1,"{'commodityCode': 601, 'countryCode': 2010, 'w...","{'commodityCode': 601, 'countryCode': 2, 'week...","{'commodityCode': 601, 'countryCode': 2010, 'w...","{'commodityCode': 601, 'countryCode': 2, 'week...","{'commodityCode': 601, 'countryCode': 2010, 'w...","{'commodityCode': 601, 'countryCode': 2, 'week...","{'commodityCode': 601, 'countryCode': 2010, 'w...","{'commodityCode': 601, 'countryCode': 2, 'week...","{'commodityCode': 601, 'countryCode': 2010, 'w...","{'commodityCode': 601, 'countryCode': 2, 'week...",...,None,None,None,None,None,None,None,None,None,None
2,"{'commodityCode': 1701, 'countryCode': 1220, '...","{'commodityCode': 1701, 'countryCode': 2010, '...","{'commodityCode': 1701, 'countryCode': 2050, '...","{'commodityCode': 1701, 'countryCode': 2150, '...","{'commodityCode': 1701, 'countryCode': 2230, '...","{'commodityCode': 1701, 'countryCode': 2250, '...","{'commodityCode': 1701, 'countryCode': 2360, '...","{'commodityCode': 1701, 'countryCode': 2410, '...","{'commodityCode': 1701, 'countryCode': 2430, '...","{'commodityCode': 1701, 'countryCode': 2470, '...",...,None,None,None,None,None,None,None,None,None,None
3,"{'commodityCode': 1702, 'countryCode': 1220, '...","{'commodityCode': 1702, 'countryCode': 2010, '...","{'commodityCode': 1702, 'countryCode': 2050, '...","{'commodityCode': 1702, 'countryCode': 2110, '...","{'commodityCode': 1702, 'countryCode': 2150, '...","{'commodityCode': 1702, 'countryCode': 2190, '...","{'commodityCode': 1702, 'countryCode': 2230, '...","{'commodityCode': 1702, 'countryCode': 2250, '...","{'commodityCode': 1702, 'countryCode': 2470, '...","{'commodityCode': 1702, 'countryCode': 2480, '...",...,None,None,None,None,None,None,None,None,None,None
4,"{'commodityCode': 107, 'countryCode': 1220, 'w...","{'commodityCode': 107, 'countryCode': 2010, 'w...","{'commodityCode': 107, 'countryCode': 2050, 'w...","{'commodityCode': 107, 'countryCode': 2080, 'w...","{'commodityCode': 107, 'countryCode': 2110, 'w...","{'commodityCode': 107, 'countryCode': 2150, 'w...","{'commodityCode': 107, 'countryCode': 2190, 'w...","{'commodityCode': 107, 'countryCode': 2230, 'w...","{'commodityCode': 107, 'countryCode': 2250, 'w...","{'commodityCode': 107, 'countryCode': 2410, 'w...",...,None,None,None,None,None,None,None,None,None,None
5,"{'commodityCode': 601, 'countryCode': 2010, 'w...","{'commodityCode': 601, 'countryCode': 2, 'week...","{'commodityCode': 601, 'countryCode': 2010, 'w...","{'commodityCode': 601, 'countryCode': 2, 'week...","{'commodityCode': 601, 'countryCode': 2010, 'w...","{'commodityCode': 601, 'countryCode': 2, 'week...","{'commodityCode': 601, 'countryCode': 2010, 'w...","{'commodityCode': 601, 'countryCode': 2, 'week...","{'commodityCode': 601, 'countryCode': 2010, 'w...","{'commodityCode': 601, 'countryCode': 2, 'week...",...,None,None,None,None,None,None,None,None,None,None
6,"{'commodityCode': 1701, 'countryCode': 1220, '...","{'commodityCode': 1701, 'countryCode': 2010, '...","{'commodityCode': 1701, 'countryCode': 2050, '...","{'commodityCode': 1701, 'countryCode': 2110, '...","{'commodityCode': 1701, 'countryCode': 2150, '...","{'commodityCode': 1701, 'countryCode': 2230, '...","{'commodityCode': 1701, 'countryCode': 2250, '...","{'commodityCode': 1701, 'countryCode': 2320, '...","{'commodityCode': 1701, 'countryCode': 2360, '..."

In [92]:
response.json()

[{'commodityCode': 1702,
  'countryCode': 1220,
  'weeklyExports': 1476,
  'accumulatedExports': 1476,
  'outstandingSales': 20212,
  'grossNewSales': 1990,
  'currentMYNetSales': 1670,
  'currentMYTotalCommitment': 21688,
  'nextMYOutstandingSales': 0,
  'nextMYNetSales': 0,
  'unitId': 1,
  'weekEndingDate': '2021-01-07T00:00:00'},
 {'commodityCode': 1702,
  'countryCode': 2010,
  'weeklyExports': 6920,
  'accumulatedExports': 6920,
  'outstandingSales': 99185,
  'grossNewSales': 7249,
  'currentMYNetSales': 6569,
  'currentMYTotalCommitment': 106105,
  'nextMYOutstandingSales': 0,
  'nextMYNetSales': 0,
  'unitId': 1,
  'weekEndingDate': '2021-01-07T00:00:00'},
 {'commodityCode': 1702,
  'countryCode': 2050,
  'weeklyExports': 182,
  'accumulatedExports': 182,
  'outstandingSales': 2697,
  'grossNewSales': 270,
  'currentMYNetSales': 190,
  'currentMYTotalCommitment': 2879,
  'nextMYOutstandingSales': 0,
  'nextMYNetSales': 0,
  'unitId': 1,
  'weekEndingDate': '2021-01-07T00:00:00'

In [66]:
headers = {
    'accept': 'application/json',
    'X-Api-Key': 'lx8NMObcTL9PSZSCZzJyykInlqLEx5WZZKtbGITw',
}

In [75]:
url = "https://api.fas.usda.gov/api/esr/exports/commodityCode/601/allCountries/marketYear/2023"

In [76]:


response = requests.get(url = url, headers=headers)

In [77]:
response.json()

[{'commodityCode': 601,
  'countryCode': 2010,
  'weeklyExports': 0,
  'accumulatedExports': 0,
  'outstandingSales': 0,
  'grossNewSales': 0,
  'currentMYNetSales': 0,
  'currentMYTotalCommitment': 0,
  'nextMYOutstandingSales': 0,
  'nextMYNetSales': 0,
  'unitId': 1,
  'weekEndingDate': '2022-06-09T00:00:00'},
 {'commodityCode': 601,
  'countryCode': 2,
  'weeklyExports': 0,
  'accumulatedExports': 0,
  'outstandingSales': 0,
  'grossNewSales': 0,
  'currentMYNetSales': 0,
  'currentMYTotalCommitment': 0,
  'nextMYOutstandingSales': 0,
  'nextMYNetSales': 0,
  'unitId': 1,
  'weekEndingDate': '2022-06-09T00:00:00'},
 {'commodityCode': 601,
  'countryCode': 2010,
  'weeklyExports': 0,
  'accumulatedExports': 0,
  'outstandingSales': 0,
  'grossNewSales': 0,
  'currentMYNetSales': 0,
  'currentMYTotalCommitment': 0,
  'nextMYOutstandingSales': 0,
  'nextMYNetSales': 0,
  'unitId': 1,
  'weekEndingDate': '2022-06-16T00:00:00'},
 {'commodityCode': 601,
  'countryCode': 2,
  'weeklyExpor

In [63]:
response.json()

[{'commodityCode': 101, 'commodityName': 'Wheat - HRW', 'unitId': 1},
 {'commodityCode': 102, 'commodityName': 'Wheat - SRW', 'unitId': 1},
 {'commodityCode': 103, 'commodityName': 'Wheat - HRS', 'unitId': 1},
 {'commodityCode': 104, 'commodityName': 'Wheat - White', 'unitId': 1},
 {'commodityCode': 105, 'commodityName': 'Wheat - Durum', 'unitId': 1},
 {'commodityCode': 106, 'commodityName': 'Wheat - Mixed', 'unitId': 1},
 {'commodityCode': 107, 'commodityName': 'All Wheat', 'unitId': 1},
 {'commodityCode': 201, 'commodityName': 'Wheat Products', 'unitId': 1},
 {'commodityCode': 301, 'commodityName': 'Barley', 'unitId': 1},
 {'commodityCode': 401, 'commodityName': 'Corn', 'unitId': 1},
 {'commodityCode': 501, 'commodityName': 'Rye', 'unitId': 1},
 {'commodityCode': 601, 'commodityName': 'Oats', 'unitId': 1},
 {'commodityCode': 701, 'commodityName': 'Sorghum', 'unitId': 1},
 {'commodityCode': 801, 'commodityName': 'Soybeans', 'unitId': 1},
 {'commodityCode': 901, 'commodityName': 'Soybe

In [71]:
countries_response = requests.get("https://api.fas.usda.gov/api/esr/countries", headers=headers)
countries_response.json()

[{'countryCode': 1,
  'countryName': 'EUROPEAN',
  'countryDescription': 'EUROPEAN UNION - 27           ',
  'regionId': 1,
  'gencCode': None},
 {'countryCode': 2,
  'countryName': 'UNKNOWN',
  'countryDescription': 'UNKNOWN',
  'regionId': 99,
  'gencCode': 'AX1'},
 {'countryCode': 1010,
  'countryName': 'GREENLD ',
  'countryDescription': 'GREENLAND                      ',
  'regionId': 11,
  'gencCode': 'GRL'},
 {'countryCode': 1220,
  'countryName': 'CANADA  ',
  'countryDescription': 'CANADA                         ',
  'regionId': 11,
  'gencCode': 'CAN'},
 {'countryCode': 1610,
  'countryName': 'MIGUEL  ',
  'countryDescription': 'ST. PIERRE AND MIQUELON        ',
  'regionId': 11,
  'gencCode': None},
 {'countryCode': 2010,
  'countryName': 'MEXICO  ',
  'countryDescription': 'MEXICO                         ',
  'regionId': 11,
  'gencCode': 'MEX'},
 {'countryCode': 2050,
  'countryName': 'GUATMAL ',
  'countryDescription': 'GUATEMALA                      ',
  'regionId': 11,


In [72]:
## now let's put into get requests
## we check the response status code
response = requests.get(url = com_url, headers = headers)
response.json()

'Bad API Key'

### Get endpoint and test out on a single commodity


In [ ]:
## your end point here


In [ ]:
## now let's put into get requests
## we check the response status code


In [ ]:
## let's store our response into an object called data


In [ ]:
## convert that list of dicts into a dataframe called df


In [ ]:
## Now iterate through all our target items



In [ ]:
## endpoint templates



In [ ]:
## iterate to get all the data


In [ ]:
## call list


In [ ]:
## concat into single df


In [ ]:
## call df


In [ ]:
## confirm we have all our target commodities


### 3. <a id="federal"></a>Federal Register – **tapping search terms.**

We have decades of <a href="https://docs.google.com/spreadsheets/d/130WeumbMZjcoRP4D-1uJ7bM0aKBZzt4N/edit?usp=sharing&ouid=112307892189798608417&rtpof=true&sd=true">SBA Excel files</a> that detail loans given to small businesses to recover after climate disasters. The only information we have about the type of disasters are codes in one of the columns that look like:

- CA-00279
- IL-00051
- NC-00099
- CA-00288
- LA-00079

The <a href="https://www.federalregister.gov/">Federal Register</a> allows us to search for what these codes stand for. But we can't search for nearly a thousand such disaster codes. When we try to scrape the site, it warns us to use the API instead.

Federal Register <a href="https://www.federalregister.gov/developers/documentation/api/v1#/Federal%20Register%20Documents/get_documents__format_">API documentation</a>

## find the end point

https://www.federalregister.gov/api/v1/documents.json?per_page=20&conditions[docket_id]=LA-00079

https://www.federalregister.gov/api/v1/documents.json?per_page=20&conditions[docket_id]=PA-00115

#### Test on single endpoint after figuring out how to build API call

In [1]:
## endpoint
endpoint = "https://www.federalregister.gov/api/v1/documents.json?per_page=20&conditions[docket_id]=PA-00115"

In [3]:
## get data
response = requests.get(endpoint)
data = response.json()
data

{'count': 1,
 'description': 'Documents filed under agency docket PA-00115',
 'total_pages': 1,
 'results': [{'title': 'Administrative Declaration of a Disaster for the Commonwealth of Pennsylvania',
   'type': 'Notice',
   'abstract': 'This is a notice of an Administrative declaration of a disaster for the State of Pennsylvania dated 10/6/2021. Incident: Remnants of Tropical Depression Ida. Incident Period: 09/01/2021 through 09/03/2021.',
   'document_number': '2021-22189',
   'html_url': 'https://www.federalregister.gov/documents/2021/10/13/2021-22189/administrative-declaration-of-a-disaster-for-the-commonwealth-of-pennsylvania',
   'pdf_url': 'https://www.govinfo.gov/content/pkg/FR-2021-10-13/pdf/2021-22189.pdf',
   'public_inspection_pdf_url': 'https://public-inspection.federalregister.gov/2021-22189.pdf?1634042723',
   'publication_date': '2021-10-13',
   'agencies': [{'raw_name': 'SMALL BUSINESS ADMINISTRATION',
     'name': 'Small Business Administration',
     'id': 468,
     

In [4]:
## length
len(data)

4

In [5]:
## what type is it?
type(data)

dict

In [6]:
data.get("count")

1

In [7]:
data.get("results")

[{'title': 'Administrative Declaration of a Disaster for the Commonwealth of Pennsylvania',
  'type': 'Notice',
  'abstract': 'This is a notice of an Administrative declaration of a disaster for the State of Pennsylvania dated 10/6/2021. Incident: Remnants of Tropical Depression Ida. Incident Period: 09/01/2021 through 09/03/2021.',
  'document_number': '2021-22189',
  'html_url': 'https://www.federalregister.gov/documents/2021/10/13/2021-22189/administrative-declaration-of-a-disaster-for-the-commonwealth-of-pennsylvania',
  'pdf_url': 'https://www.govinfo.gov/content/pkg/FR-2021-10-13/pdf/2021-22189.pdf',
  'public_inspection_pdf_url': 'https://public-inspection.federalregister.gov/2021-22189.pdf?1634042723',
  'publication_date': '2021-10-13',
  'agencies': [{'raw_name': 'SMALL BUSINESS ADMINISTRATION',
    'name': 'Small Business Administration',
    'id': 468,
    'url': 'https://www.federalregister.gov/agencies/small-business-administration',
    'json_url': 'https://www.federalre

In [8]:
## type
type(data.get("results"))

list

In [10]:
len((data.get("results")))

1

In [11]:
data.get("results")[0]

{'title': 'Administrative Declaration of a Disaster for the Commonwealth of Pennsylvania',
 'type': 'Notice',
 'abstract': 'This is a notice of an Administrative declaration of a disaster for the State of Pennsylvania dated 10/6/2021. Incident: Remnants of Tropical Depression Ida. Incident Period: 09/01/2021 through 09/03/2021.',
 'document_number': '2021-22189',
 'html_url': 'https://www.federalregister.gov/documents/2021/10/13/2021-22189/administrative-declaration-of-a-disaster-for-the-commonwealth-of-pennsylvania',
 'pdf_url': 'https://www.govinfo.gov/content/pkg/FR-2021-10-13/pdf/2021-22189.pdf',
 'public_inspection_pdf_url': 'https://public-inspection.federalregister.gov/2021-22189.pdf?1634042723',
 'publication_date': '2021-10-13',
 'agencies': [{'raw_name': 'SMALL BUSINESS ADMINISTRATION',
   'name': 'Small Business Administration',
   'id': 468,
   'url': 'https://www.federalregister.gov/agencies/small-business-administration',
   'json_url': 'https://www.federalregister.gov/ap

In [12]:
## targeting incidents
data.get("results")[0].get("abstract")

'This is a notice of an Administrative declaration of a disaster for the State of Pennsylvania dated 10/6/2021. Incident: Remnants of Tropical Depression Ida. Incident Period: 09/01/2021 through 09/03/2021.'

### Iterate through entire list of codes

In [13]:
## Normally will take from df as a list
## build disaster code list
disaster_codes = ["CA-00279","IL-00051", "NC-00099", "CA-00288", "LA-00079"]

In [14]:
## provide base url
base_url = "https://www.federalregister.gov/api/v1/documents.json?per_page=20&conditions[docket_id]="

In [24]:
broken_endpoints = [] ## track broken endpoints
incidents_list = []

## iterate through all endpoints
for disaster_code in disaster_codes:
    endpoint = base_url + disaster_code
    try:
        response = requests.get(endpoint)
        data = response.json()
        incident_text = data.get("results")[0].get("abstract")
        incidents_list.append({"disaster_code": disaster_code,
                              "description": incident_text})
    except:
        broken_endpoints.append(endpoint)

In [25]:
## call list
incidents_list

[{'disaster_code': 'CA-00279',
  'description': 'This is an amendment of the Presidential declaration of a major disaster for the State of California (FEMA-4344-DR), dated 10/12/ 2017. Incident: Wildfires. Incident Period: 10/08/2017 through 10/31/2017.'},
 {'disaster_code': 'IL-00051',
  'description': 'This is a notice of an Administrative declaration of a disaster for the State of Illinois dated 05/07/2018. Incident: Severe Storms and Flooding. Incident Period: 02/14/2018 through 03/04/2018.'},
 {'disaster_code': 'NC-00099',
  'description': 'This is an amendment of the Presidential declaration of a major disaster for the State of North Carolina (FEMA-4393-DR), dated 09/14/2018. Incident: Hurricane Florence. Incident Period: 09/07/2018 through 09/29/2018.'},
 {'disaster_code': 'CA-00288',
  'description': 'This is an amendment of the Presidential declaration of a major disaster for the State of California (FEMA-4382-DR), dated 08/04/ 2018. Incident: Wildfires and High Winds. Inciden

In [26]:
pd.DataFrame(incidents_list)

,disaster_code,description
0,CA-00279,This is an amendment of the Presidential decla...
1,IL-00051,This is a notice of an Administrative declarat...
2,NC-00099,This is an amendment of the Presidential decla...
3,CA-00288,This is an amendment of the Presidential decla...
4,LA-00079,This is a notice of an Administrative declarat...


In [33]:
actual_incidents = []
for item in incidents_list:
    print(item.get("description"))
    try:
        incident_text = \
        item["description"].split("Incident:")[1].split(". Incident Period:")[0].strip()

SyntaxError: unexpected EOF while parsing (1784546479.py, line 6)

In [ ]:
## your target endpoint


### 4. Energy Information Administration – **dealing with pagination.**

From the <a href="https://www.eia.gov/">Energy Information Administration</a>, we want to compile energy generation by type of fuel and region for about 5 days.

We will encounter a limit on the number of items per API call.

Find our API endpoint first.

In [ ]:
## get response


In [ ]:
## import ceiling division from math



In [ ]:
## paginate our API calls



In [ ]:
## for loop with pagination


In [ ]:
## call all data


In [ ]:
## length


In [ ]:
## use itertools to flatten list with nested lists


In [ ]:
## flatten nested lists


In [ ]:
## convert to df


In [ ]:
fips_codes = {
    'Alabama': '01',
    'Alaska': '02',
    'Arizona': '04',
    'Arkansas': '05',
    'California': '06',
    'Colorado': '08',
    'Connecticut': '09',
    'Delaware': '10',
    'Florida': '12',
    'Georgia': '13',
    'Hawaii': '15',
    'Idaho': '16',
    'Illinois': '17',
    'Indiana': '18',
    'Iowa': '19',
    'Kansas': '20',
    'Kentucky': '21',
    'Louisiana': '22',
    'Maine': '23',
    'Maryland': '24',
    'Massachusetts': '25',
    'Michigan': '26',
    'Minnesota': '27',
    'Mississippi': '28',
    'Missouri': '29',
    'Montana': '30',
    'Nebraska': '31',
    'Nevada': '32',
    'New Hampshire': '33',
    'New Jersey': '34',
    'New Mexico': '35',
    'New York': '36',
    'North Carolina': '37',
    'North Dakota': '38',
    'Ohio': '39',
    'Oklahoma': '40',
    'Oregon': '41',
    'Pennsylvania': '42',
    'Rhode Island': '44',
    'South Carolina': '45',
    'South Dakota': '46',
    'Tennessee': '47',
    'Texas': '48',
    'Utah': '49',
    'Vermont': '50',
    'Virginia': '51',
    'Washington': '53',
    'West Virginia': '54',
    'Wisconsin': '55',
    'Wyoming': '56',
    'District of Columbia': '11',
    'Puerto Rico': '72'
}
